In [14]:
import pandas as pd

In [15]:
d = pd.read_excel("D:\\Agreggate Table Project.xlsx")

In [16]:
d.head(1)

,ClaimID,FirstName,LastName,Email,Gender,Address,Charges,Payments,Refunds,ContractualAdjustments,ServiceDate,DOB
0,1,Marijn,Tappin,mtappin0@sina.com.cn,Male,Apt 241,98929.1,21078.37,414.15,991.32,11/28/2022,8/9/2011


In [17]:
#Create Agregate Table
DAgg = d.groupby('ServiceDate').agg({
    'Charges': 'sum',
    'Payments': 'sum',
    'Refunds': 'sum',
    'ContractualAdjustments': 'sum'
}).reset_index()

In [22]:
DAgg.head(3)

,ServiceDate,Charges,Payments,Refunds,ContractualAdjustments,SDMonth,SDYear,NCR,Rolling30dayCharges
0,2022-01-01,91275.93,25955.75,643.04,366.64,January,2022,0.278439,91275.93
1,2022-01-02,75566.17,24659.02,542.21,967.96,January,2022,0.323289,166842.10
2,2022-01-05,22275.02,49998.05,999.67,359.57,January,2022,2.235792,189117.12


In [19]:
DAgg['ServiceDate'] = pd.to_datetime(d['ServiceDate'])
DAgg['SDMonth'] = DAgg['ServiceDate'].dt.month_name()
DAgg['SDYear'] = DAgg['ServiceDate'].dt.year
DAgg = DAgg.sort_values(by='ServiceDate').reset_index(drop=True)

In [20]:
DAgg['NCR'] = (DAgg['Payments']-DAgg['Refunds'])/(DAgg['Charges']-DAgg['ContractualAdjustments'])

In [33]:
# 30 Day rolling total Fields
DAgg['Rolling30dayCharges'] = DAgg.set_index('ServiceDate')['Charges'].rolling('30D').sum().reset_index(drop=True)
DAgg['Rolling30dayPayments'] = DAgg.set_index('ServiceDate')['Payments'].rolling('30D').sum().reset_index(drop=True)
DAgg['Rolling30dayRefunds'] = DAgg.set_index('ServiceDate')['Refunds'].rolling('30D').sum().reset_index(drop=True)
DAgg['Rolling30dayContractualAdjustments'] = DAgg.set_index('ServiceDate')['ContractualAdjustments'].rolling('30D').sum().reset_index(drop=True)

# 90 Rolling Fields
DAgg['Rolling90dayCharges'] = DAgg.set_index('ServiceDate')['Charges'].rolling('90D').sum().reset_index(drop=True)
DAgg['Rolling90dayPayments'] = DAgg.set_index('ServiceDate')['Payments'].rolling('90D').sum().reset_index(drop=True)
DAgg['Rolling90dayRefunds'] = DAgg.set_index('ServiceDate')['Refunds'].rolling('90D').sum().reset_index(drop=True)
DAgg['Rolling90dayContractualAdjustments'] = DAgg.set_index('ServiceDate')['ContractualAdjustments'].rolling('90D').sum().reset_index(drop=True)

# 365 Rolling Field
DAgg['Rolling90dayCharges'] = DAgg.set_index('ServiceDate')['Charges'].rolling('365D').sum().reset_index(drop=True)
DAgg['Rolling90dayPayments'] = DAgg.set_index('ServiceDate')['Payments'].rolling('365D').sum().reset_index(drop=True)
DAgg['Rolling90dayRefunds'] = DAgg.set_index('ServiceDate')['Refunds'].rolling('365D').sum().reset_index(drop=True)
DAgg['Rolling90dayContractualAdjustments'] = DAgg.set_index('ServiceDate')['ContractualAdjustments'].rolling('365D').sum().reset_index(drop=True)

#YTD
DAgg['YTDCharges'] = DAgg.groupby('SDYear')['Charges'].transform(lambda x: x.cumsum())
DAgg['YTDPayments'] = DAgg.groupby('SDYear')['Payments'].transform(lambda x: x.cumsum())
DAgg['YTDRefunds'] = DAgg.groupby('SDYear')['Refunds'].transform(lambda x: x.cumsum())
DAgg['YTDContractualAdjustments'] = DAgg.groupby('SDYear')['ContractualAdjustments'].transform(lambda x: x.cumsum())

#MTD
DAgg['MTDCharges'] = DAgg.groupby(['SDYear', 'SDMonth'])['Charges'].transform(lambda x: x.cumsum())
DAgg['MTDPayments'] = DAgg.groupby(['SDYear', 'SDMonth'])['Payments'].transform(lambda x: x.cumsum())
DAgg['MTDRefunds'] = DAgg.groupby(['SDYear', 'SDMonth'])['Refunds'].transform(lambda x: x.cumsum())
DAgg['MTDContractualAdjustments'] = DAgg.groupby(['SDYear', 'SDMonth'])['ContractualAdjustments'].transform(lambda x: x.cumsum())

#Lifetime AVG
DAgg['LifetimeAVGCharges'] = DAgg['Charges'].expanding().mean().round(2)
DAgg['LifetimeAVGPayments'] = DAgg['Payments'].expanding().mean().round(2)
DAgg['LifetimeAVGRefunds'] = DAgg['Refunds'].expanding().mean().round(2)
DAgg['LifetimeAVGContractualAdjustments'] = DAgg['ContractualAdjustments'].expanding().mean().round(2)

In [34]:
DAgg.head(3)

,ServiceDate,Charges,Payments,Refunds,ContractualAdjustments,SDMonth,SDYear,NCR,Rolling30dayCharges,Rolling30dayPayments,...,YTDRefunds,YTDContractualAdjustments,MTDCharges,MTDPayments,MTDRefunds,MTDContractualAdjustments,LifetimeAVGCharges,LifetimeAVGPayments,LifetimeAVGRefunds,LifetimeAVGContractualAdjustments
0,2022-01-01,91275.93,25955.75,643.04,366.64,January,2022,0.278439,91275.93,25955.75,...,643.04,366.64,91275.93,25955.75,643.04,366.64,91275.93,25955.75,643.04,366.64
1,2022-01-02,75566.17,24659.02,542.21,967.96,January,2022,0.323289,166842.10,50614.77,...,1185.25,1334.60,166842.10,50614.77,1185.25,1334.60,83421.05,25307.38,592.62,667.30
2,2022-01-05,22275.02,49998.05,999.67,359.57,January,2022,2.235792,189117.12,100612.82,...,2184.92,1694.17,189117.12,100612.82,2184.92,1694.17,63039.04,33537.61,728.31,564.72
